In [1]:
import pyspark
import pandas as pd
import numpy as np

In [2]:
# Setting Spark environments

import os

os.environ['SPARK_HOME'] = '/Users/atharvakarnik/spark-3.5.1-bin-hadoop3'
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'

In [3]:
# Creating a literally (not truely though) random dataframe

rows = np.random.randint(10, 13)
data = {
    "Temperature (°C)": np.random.uniform(20, 35, size=rows).round(2),
    "Humidity (%)": np.random.uniform(30, 90, size=rows).round(2)
}

df = pd.DataFrame(data)

display(df)

,Temperature (°C),Humidity (%)
0,24.17,87.14
1,20.10,79.84
2,21.42,32.56
3,26.46,57.69
4,33.14,83.88
5,24.06,43.50
6,28.13,84.36
7,28.22,56.82
8,21.99,81.11
9,26.28,61.11


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('RandomApp').getOrCreate()

24/08/11 02:19:19 WARN Utils: Your hostname, Atharvas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.184.68.157 instead (on interface en0)
24/08/11 02:19:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/11 02:19:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [6]:
spark_df = spark.createDataFrame(df)

In [7]:
spark_df.show()

+----------------+------------+
|Temperature (°C)|Humidity (%)|
+----------------+------------+
|           24.17|       87.14|
|            20.1|       79.84|
|           21.42|       32.56|
|           26.46|       57.69|
|           33.14|       83.88|
|           24.06|        43.5|
|           28.13|       84.36|
|           28.22|       56.82|
|           21.99|       81.11|
|           26.28|       61.11|
+----------------+------------+



24/08/11 02:19:30 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [15]:
spark_df.printSchema()

root
 |-- Temperature (°C): double (nullable = true)
 |-- Humidity (%): double (nullable = true)



In [16]:
spark_df.select(['Temperature (°C)']).show()

+----------------+
|Temperature (°C)|
+----------------+
|           24.17|
|            20.1|
|           21.42|
|           26.46|
|           33.14|
|           24.06|
|           28.13|
|           28.22|
|           21.99|
|           26.28|
+----------------+



In [17]:
# To end the session, let's be kind on our kernel :)

just_confirming = input("Hold up! Do you really want to end Spark sessio? 'Y'/'N'")
if just_confirming == 'Y': 
    print("Alright, ending Spark Session :'(")
    spark.stop()
elif just_confirming == 'N': print("That was a close call :)")
else: print("You may want to read code before executig ~_~")

Hold up! Do you really want to end Spark sessio? 'Y'/'N' Y


Alright, ending Spark Session :'(
